In [1]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [3]:
import numpy as np
import torch
import string
import os
from tqdm import tqdm
import torch.nn as nn
from torch.nn import functional as F
import datetime
from mingpt.md import MemData
from mingpt.marker_dataset import MarkerDataset
from mingpt.math_dataset import MathDataset
from mingpt.model import GPT, GPTConfig, GPT1Config
from torch.utils.data.dataloader import DataLoader
from mingpt.trainer import Trainer, TrainerConfig
from mingpt.examiner import Examiner
%load_ext autoreload
%autoreload 2

In [4]:
#create a dataset
!rm -rf run models
!cp -r data run
!mkdir models
fn_data = 'run/numbers__list_prime_factors.txt'

In [5]:
# Add memory data structure to training data
memory_slots = 7
MD = MemData(memory_slots)
MD.initiate_mem_slot_data(fn_data)

In [6]:
fn_test = 'run/test_numbers__list_prime_factors.txt'
fn_train = 'run/train_numbers__list_prime_factors.txt'
train_dataset = MathDataset(fname=fn_train, MD=MD)

In [7]:
print(MD.block_size)
print(MD.vocab_size)
print(MD.max_trg)

260
102
27


In [8]:
# initialize a baby GPT model
mconf = GPTConfig(MD.vocab_size, MD.block_size,
                  n_layer=4, n_head=8, n_embd=256)
model = GPT(mconf)

12/17/2020 05:09:49 - INFO - mingpt.model -   number of parameters: 3.278336e+06


In [9]:
max_it = 100
main_epoch = 1
marker_epoch = 1
current_it = 0

exp_folder = 'models/' + datetime.datetime.now().strftime('%Y-%m-%d~%H:%M:%S')
examiner = Examiner(MD)

while(current_it < max_it):

    # Switch between main training and marker training
    if current_it % 2 == 0:
        print("Loading Main Dataset\n")
        train_dataset = MathDataset(fname=fn_train, MD=MD)
        test_dataset = MathDataset(fname=fn_test, MD=MD)
        epoch = main_epoch
    else:
        print("Loading Marker Dataset\n")
        train_dataset = MarkerDataset(fname=fn_train, MD=MD)
        test_dataset = MarkerDataset(fname=fn_test, MD=MD)
        epoch = marker_epoch
    
    # Trainer Config
    tconf = TrainerConfig(max_epochs=epoch, batch_size=256, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=1024, final_tokens=epoch*len(train_dataset)*(MD.vocab_size+1),
                      num_workers=0)
    
    # Create the first training round
    print("Training: ", str(current_it))
    trainer = Trainer(model, train_dataset, test_dataset, tconf)
    #trainer.train()
    trainer.save_checkpoint(exp_folder, str(current_it))
    
    # Examine the model and create new dataset
    if current_it % 2 == 0:
        print("Exam and new dataset-------------\n")
        print("Training exam \n")
        examiner.exam(fn_train, train_dataset, trainer, 5000)
        print("Test exam \n")
        examiner.exam(fn_test, test_dataset, trainer, 5000)
    
    current_it += 1

Loading Main Dataset

Training:  0


12/17/2020 05:09:50 - INFO - mingpt.trainer -   saving models/2020-12-17~05:09:49/0.pth
Iiter 2:   0%|          | 0/900 [00:00<?, ?it/s]

Exam and new dataset-------------

Training exam 



Iiter 17:  14%|█▍        | 14/100 [00:00<00:00, 139.74it/s]

Final score: 0/784 = 0.00% correct
Saving new files to disk...
Test exam 



...

torch.Size([1, 178])
torch.Size([19, 176])
tensor([[61, 84, 94, 95,  7, 95, 83, 80,  7, 91, 93, 84, 88, 80,  7, 81, 76, 78,
         95, 90, 93, 94,  7, 90, 81,  7, 42, 44, 44, 46, 43, 41, 44, 40, 21,  1,
         73, 73, 73, 73, 34, 73, 34, 73, 73, 34, 73, 34, 73, 34, 73, 73, 34, 73,
         34,  2, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 34, 73,
         34, 73, 34,  2, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73,
         73, 73, 73, 73, 73,  2, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
         15, 15, 95, 95, 95, 95, 95,  2, 95, 95, 15, 21, 15, 21, 15, 21, 15, 21,
         15, 21, 15, 21, 15, 21, 15, 21, 15,  2, 95, 95, 44, 95, 21, 59, 38, 98,
         44, 15, 21, 59, 38, 98, 44, 15, 21, 15, 21,  2,  2,  3, 42, 19,  7, 43,
         19,  7, 45, 19,  7, 41, 43, 19,  7, 47, 49, 19,  7, 43, 49, 47]])
tensor([[61, 84, 94,  ...,  0,  0,  0],
        [61, 84, 94,  ...,  0,  0,  0],
        [61, 84, 94,  ...,  0,  0,  0],
        ...,
        [61, 84, 94,

Iiter 0: 100%|██████████| 1/1 [00:00<00:00, 1392.53it/s]

torch.Size([1, 188])
torch.Size([1, 191])
tensor([[61, 84, 94, 95,  7, 95, 83, 80,  7, 91, 93, 84, 88, 80,  7, 81, 76, 78,
         95, 90, 93, 94,  7, 90, 81,  7, 48, 42, 49, 41, 45, 43, 46, 21,  1, 21,
         15, 21, 15, 21, 15, 21, 15, 21, 15, 21, 15, 21, 15, 21, 15, 21, 15, 21,
          2, 21, 59, 38, 21, 59, 38, 21, 59, 38, 21, 59, 38, 21, 59, 38, 21, 59,
         38, 21,  2, 38, 59, 38, 59, 38, 21, 59, 38, 21, 59, 38, 21, 59, 38, 21,
         59, 38, 21, 59,  2, 95, 44, 99, 95, 44, 99, 95, 44, 99, 95, 99, 95, 99,
         95, 99, 95, 99, 95, 99,  2, 95, 44, 99, 44, 99, 44, 99, 44, 99, 95, 44,
         99, 95, 44, 99, 95, 44, 99, 95,  2, 15, 15, 15, 15, 15, 15, 15, 15, 15,
         15, 15, 15, 15, 15, 15, 15, 15, 15, 15,  2, 95, 44, 95, 44, 98, 44, 98,
         98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98, 98,  2,  3, 42, 19,  7, 41,
         41, 19,  7, 44, 47, 41, 41, 41]])
tensor([[61, 84, 94, 95,  7, 95, 83, 80,  7, 91, 93, 84, 88, 80,  7, 81, 76, 78,
         95, 90, 93, 94,


/usr/lib/python3/dist-packages/numpy/core/fromnumeric.py:3256: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/lib/python3/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: max() arg is an empty sequence

In [ ]:
test = list(range(10))

In [ ]:
test[:20]

In [ ]:
test[3:]

In [ ]:
test = ['', '', 'cat', '']
for i in test:
    print(i)

In [ ]:
sum([1, 2, 3, ])

In [ ]:
X = torch.tensor([0.1, 0.5, -1.0, 0, 1.2, 0])

In [ ]:
X = X[X > 0.1]

In [ ]:
X